In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)

## Loading dataset

In [1]:
!git clone https://github.com/hieunghia-pat/StudentPerformance

Cloning into 'StudentPerformance'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 4), reused 13 (delta 4), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip


Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
To: /content/student.zip
100% 20.5k/20.5k [00:00<00:00, 326kB/s]


In [3]:
!unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("student-mat.csv", sep=";")

In [6]:
data.sample(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
281,GP,M,17,U,LE3,A,3,2,teacher,services,home,mother,1,1,1,no,no,no,no,yes,yes,yes,no,4,4,4,3,4,3,19,11,9,10
203,GP,F,17,R,GT3,T,2,2,other,other,reputation,mother,1,1,0,no,yes,no,no,yes,yes,yes,no,5,3,2,1,2,3,18,7,6,6
37,GP,M,16,R,GT3,A,4,4,other,teacher,reputation,mother,2,3,0,no,yes,no,yes,yes,yes,yes,yes,2,4,3,1,1,5,7,15,16,15
59,GP,F,16,U,GT3,T,4,2,services,other,course,mother,1,2,0,no,yes,no,no,yes,yes,yes,no,4,2,3,1,1,5,2,15,16,16
191,GP,F,17,U,GT3,T,1,1,at_home,services,course,mother,1,2,0,no,no,no,yes,yes,yes,yes,no,5,3,3,1,1,3,0,8,8,9
68,GP,F,15,R,LE3,T,2,2,health,services,reputation,mother,2,2,0,yes,yes,yes,no,yes,yes,yes,no,4,1,3,1,3,4,2,8,9,8
129,GP,M,16,R,GT3,T,4,4,teacher,teacher,course,mother,1,1,0,no,no,yes,yes,yes,yes,yes,no,3,5,5,2,5,4,8,18,18,18
87,GP,F,15,U,GT3,T,4,2,other,other,reputation,mother,1,3,0,no,yes,no,yes,yes,yes,yes,no,5,3,3,1,3,1,4,13,14,14
187,GP,M,16,U,LE3,T,2,1,other,other,course,mother,1,2,0,no,no,yes,yes,yes,yes,yes,yes,4,2,3,1,2,5,0,15,15,15
36,GP,M,15,U,LE3,T,4,3,teacher,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,no,5,4,3,1,1,4,2,15,16,18


In [7]:
data.shape

(395, 33)

### Load data and preprocess

In [8]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv("student-mat.csv", sep=";")

In [10]:
data.sample(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
317,GP,F,18,U,GT3,T,4,3,other,other,course,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,4,3,4,1,1,5,9,9,10,9
5,GP,M,16,U,LE3,T,4,3,services,other,reputation,mother,1,2,0,no,yes,yes,yes,yes,yes,yes,no,5,4,2,1,2,5,10,15,15,15
376,MS,F,20,U,GT3,T,4,2,health,other,course,other,2,3,2,no,yes,yes,no,no,yes,yes,yes,5,4,3,1,1,3,4,15,14,15
357,MS,F,17,U,LE3,A,3,2,services,other,reputation,mother,2,2,0,no,no,no,no,yes,yes,no,yes,1,2,3,1,2,5,2,12,12,11
301,GP,M,17,U,LE3,T,4,4,other,teacher,home,father,2,1,0,no,no,yes,no,yes,yes,yes,no,4,1,1,2,2,5,0,11,11,10
13,GP,M,15,U,GT3,T,4,3,teacher,other,course,mother,2,2,0,no,yes,yes,no,yes,yes,yes,no,5,4,3,1,2,3,2,10,10,11
161,GP,M,15,R,GT3,T,3,2,other,other,course,mother,2,2,2,yes,yes,no,no,yes,yes,yes,yes,4,4,4,1,4,3,6,5,9,7
164,GP,M,17,R,LE3,T,1,1,other,services,course,mother,4,2,3,no,no,no,yes,yes,no,no,yes,5,3,5,1,5,5,0,5,8,7
101,GP,M,16,U,GT3,T,4,4,services,teacher,other,father,1,3,0,no,yes,no,yes,yes,yes,yes,yes,4,4,3,1,1,4,0,16,17,17
129,GP,M,16,R,GT3,T,4,4,teacher,teacher,course,mother,1,1,0,no,no,yes,yes,yes,yes,yes,no,3,5,5,2,5,4,8,18,18,18


In [11]:
features = data.columns[:-3]

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
encoders = {}
x = []
for feature in features:
    if data[feature].dtype == "object":
        encoders[feature] = OneHotEncoder(sparse=False)
        encoded_feature = encoders[feature].fit_transform(data[feature].values.reshape(-1, 1))
    else:
        encoded_feature = data[feature].values.reshape(-1, 1)
    x.append(encoded_feature)

x = np.concatenate(x, axis=1)
y = data["G3"].values

### Splitting dataset

In [14]:
from sklearn.model_selection import StratifiedKFold

In [15]:
folds = StratifiedKFold(n_splits=10, shuffle=True)

### Linear model for all attributes

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
linear_models = {}
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  ln_model = Pipeline([
                     ("scaler", StandardScaler()),
                     ("linear_model", LinearRegression())
  ])

  ln_model.fit(x_train, y_train)
  y_pred = ln_model.predict(x_test)
  
  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

  linear_models[f"Stage_{ith}"] = ln_model

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.065255,21.219212
1,0,0.338618,14.808348
2,0,0.246786,17.209061
3,0,0.091090,21.086152
4,0,0.202251,16.750743
5,0,0.010525,18.007021
6,0,0.256416,15.740907
7,0,-0.251734,25.153183
8,0,-0.139478,22.782059
9,0,-0.179498,23.582210


### Linear model for all attributes with all interactions

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
linear_models = {}
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  ln_model = Pipeline([
                     ("scaler", StandardScaler()),
                     ("interactive_feature", PolynomialFeatures(interaction_only=True, include_bias=True, degree=2)),
                     ("linear_model", LinearRegression())
  ])

  ln_model.fit(x_train, y_train)
  y_pred = ln_model.predict(x_test)
  
  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

  linear_models[f"Stage_{ith}"] = ln_model

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-1.447394,48.750552
1,0,-1.188482,49.000110
2,0,-0.785015,40.783126
3,0,-0.784258,41.393665
4,0,-1.517386,52.858806
5,0,-1.653262,48.285526
6,0,-1.286325,48.399146
7,0,-0.892784,38.034883
8,0,-1.084907,41.684439
9,0,-1.726913,54.520340


### Polynomial model for all attributes

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
results = {
    "Poly_degree": [],
    "R2": [],
    "MSE": []
}

In [ ]:
for degree in range(1, 5):
  ith = 0
  r2_scores = []
  mse_scores = []
  for train_index, test_index in folds.split(x, y):
    x_train = x[train_index]
    y_train = y[train_index]
    
    x_test = x[test_index]
    y_test = y[test_index]

    ln_model = Pipeline([
                      ("scaler", StandardScaler()),
                      ("poly_feature", PolynomialFeatures(degree)),
                      ("linear_model", LinearRegression())
    ])

    ln_model.fit(x_train, y_train)
    y_pred = ln_model.predict(x_test)

    r2_scores.append(r2_score(y_test, y_pred))
    mse_scores.append(mean_squared_error(y_test, y_pred))
    
  results["Poly_degree"].append(degree)
  results["R2"].append(np.array(r2_scores).mean())
  results["MSE"].append(np.array(mse_scores).mean())

In [ ]:
results = pd.DataFrame(results)
results

## Random Forest model for all attributes

### Load data and preprocess

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("student-mat.csv", sep=";")

In [ ]:
data.sample(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
45,GP,F,15,U,LE3,A,4,3,other,other,course,mother,1,2,0,yes,yes,yes,yes,yes,yes,yes,yes,5,2,2,1,1,5,8,8,8,6
369,MS,F,18,R,GT3,T,4,4,other,teacher,other,father,3,2,0,no,yes,yes,no,no,yes,yes,yes,3,2,2,4,2,5,10,14,12,11
66,GP,M,15,U,GT3,A,4,4,other,services,reputation,mother,1,4,0,no,yes,no,yes,no,yes,yes,yes,1,3,3,5,5,3,4,13,13,12
91,GP,F,15,U,GT3,T,4,3,services,other,reputation,mother,1,1,0,no,no,yes,yes,yes,yes,yes,no,4,5,5,1,3,1,4,16,17,18
362,MS,F,18,U,GT3,T,3,3,services,services,other,mother,2,2,0,no,yes,no,no,yes,yes,yes,yes,4,3,2,1,3,3,0,11,11,10
104,GP,M,15,U,GT3,A,3,4,services,other,course,mother,1,2,0,no,yes,yes,yes,yes,yes,yes,no,5,4,4,1,1,1,0,16,18,18
18,GP,M,17,U,GT3,T,3,2,services,services,course,mother,1,1,3,no,yes,no,yes,yes,yes,yes,no,5,5,5,2,4,5,16,6,5,5
186,GP,M,16,U,GT3,T,1,2,services,services,other,mother,1,1,0,no,yes,yes,yes,yes,yes,yes,yes,3,3,3,1,2,3,2,11,12,11
374,MS,F,18,R,LE3,T,4,4,other,other,reputation,mother,2,3,0,no,no,no,no,yes,yes,yes,no,5,4,4,1,1,1,0,19,18,19
309,GP,F,19,U,LE3,T,1,1,at_home,other,reputation,other,1,2,1,yes,yes,no,yes,no,yes,yes,no,4,4,3,1,3,3,18,12,10,10


In [ ]:
features = data.columns[:-3]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoders = {}
x = []
for feature in features:
    if data[feature].dtype == "object":
        encoders[feature] = OneHotEncoder(sparse=False)
        encoded_feature = encoders[feature].fit_transform(data[feature].values.reshape(-1, 1))
    else:
        encoded_feature = data[feature].values.reshape(-1, 1)
    x.append(encoded_feature)

x = np.concatenate(x, axis=1)
y = data["G3"].values

### Splitting dataset

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=True)

### RF model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
n_estimators = [100, 1000]

In [ ]:
params = {
    "n_estimators": n_estimators,
}

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  grid = GridSearchCV(RandomForestRegressor(), params, refit=True, verbose=3)

  grid.fit(x_train, y_train)
  y_pred = grid.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .................n_estimators=100;, score=-0.026 total time=   0.3s
[CV 2/5] END ..................n_estimators=100;, score=0.224 total time=   0.3s
[CV 3/5] END ..................n_estimators=100;, score=0.285 total time=   0.3s
[CV 4/5] END ..................n_estimators=100;, score=0.068 total time=   0.3s
[CV 5/5] END ..................n_estimators=100;, score=0.140 total time=   0.3s
[CV 1/5] END .................n_estimators=1000;, score=0.003 total time=   3.8s
[CV 2/5] END .................n_estimators=1000;, score=0.221 total time=   3.0s
[CV 3/5] END .................n_estimators=1000;, score=0.303 total time=   4.7s
[CV 4/5] END .................n_estimators=1000;, score=0.082 total time=   4.6s
[CV 5/5] END .................n_estimators=1000;, score=0.169 total time=   4.6s
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ..................n_estimators=100;, score=0.019 total ti

In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.009596,20.110512
1,0,0.077965,20.644371
2,0,0.346106,14.939841
3,0,0.389117,14.172096
4,0,0.169495,17.438522
5,0,0.493151,9.223927
6,0,0.457798,11.477867
7,0,0.162894,16.821365
8,0,0.429673,11.402794
9,0,0.216191,15.671036


## Decision Tree model for all attributes

### Load data and preprocess

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("student-mat.csv", sep=";")

In [ ]:
data.sample(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
133,GP,F,16,U,GT3,A,3,4,services,other,course,father,1,1,0,no,no,no,no,yes,yes,yes,no,3,2,1,1,4,5,16,12,11,11
158,GP,M,16,R,GT3,T,2,2,at_home,other,course,mother,3,1,0,no,no,no,no,no,yes,no,no,4,2,2,1,2,3,2,17,15,15
218,GP,F,17,U,GT3,T,2,3,at_home,other,home,father,2,1,0,no,yes,yes,no,yes,yes,no,no,3,3,3,1,4,3,3,7,7,8
329,GP,F,17,U,GT3,T,4,4,teacher,teacher,course,mother,2,3,0,no,yes,yes,no,no,yes,yes,yes,4,3,3,1,2,4,4,14,14,14
245,GP,M,16,U,GT3,T,2,1,other,other,course,mother,3,1,0,no,no,no,no,yes,yes,yes,no,4,3,3,1,1,4,6,18,18,18
259,GP,F,17,U,LE3,T,2,2,services,services,course,father,1,4,0,no,no,yes,yes,yes,yes,yes,yes,3,4,1,1,1,2,0,10,9,0
53,GP,F,15,U,GT3,T,4,4,services,services,course,mother,1,1,0,yes,yes,yes,no,yes,yes,yes,no,3,3,4,2,3,5,0,8,10,11
390,MS,M,20,U,LE3,A,2,2,services,services,course,other,1,2,2,no,yes,yes,no,yes,yes,no,no,5,5,4,4,5,4,11,9,9,9
11,GP,F,15,U,GT3,T,2,1,services,other,reputation,father,3,3,0,no,yes,no,yes,yes,yes,yes,no,5,2,2,1,1,4,4,10,12,12
132,GP,F,17,U,LE3,T,2,2,other,other,course,father,1,1,0,no,yes,no,no,yes,yes,yes,yes,3,4,4,1,3,5,12,10,13,12


In [ ]:
features = data.columns[:-3]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoders = {}
x = []
for feature in features:
    if data[feature].dtype == "object":
        encoders[feature] = OneHotEncoder(sparse=False)
        encoded_feature = encoders[feature].fit_transform(data[feature].values.reshape(-1, 1))
    else:
        encoded_feature = data[feature].values.reshape(-1, 1)
    x.append(encoded_feature)

x = np.concatenate(x, axis=1)
y = data["G3"].values

### Splitting dataset

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=True)

### Decision Tree model

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  dc_model = DecisionTreeRegressor()

  dc_model.fit(x_train, y_train)
  y_pred = dc_model.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.213642,24.175000
1,0,-0.488388,33.325000
2,0,-0.433417,32.750000
3,0,-0.213395,28.150000
4,0,-0.747827,36.700000
5,0,-0.416004,25.769231
6,0,-0.969501,41.692308
7,0,-0.384472,27.820513
8,0,-0.442782,28.846154
9,0,0.088162,18.230769


### Extra Tree model

In [ ]:
from sklearn.tree import ExtraTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  dc_model = ExtraTreeRegressor()

  dc_model.fit(x_train, y_train)
  y_pred = dc_model.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-1.230241,44.425000
1,0,-0.110987,24.875000
2,0,-0.446548,33.050000
3,0,-0.350252,31.325000
4,0,-0.741874,36.575000
5,0,-0.875325,34.128205
6,0,-0.460774,30.923077
7,0,-0.995681,40.102564
8,0,-0.594114,31.871795
9,0,-0.041368,20.820513


## Logistic Regression model for all attributes

### Load data and preprocess

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("student-mat.csv", sep=";")

In [ ]:
data.sample(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
45,GP,F,15,U,LE3,A,4,3,other,other,course,mother,1,2,0,yes,yes,yes,yes,yes,yes,yes,yes,5,2,2,1,1,5,8,8,8,6
369,MS,F,18,R,GT3,T,4,4,other,teacher,other,father,3,2,0,no,yes,yes,no,no,yes,yes,yes,3,2,2,4,2,5,10,14,12,11
66,GP,M,15,U,GT3,A,4,4,other,services,reputation,mother,1,4,0,no,yes,no,yes,no,yes,yes,yes,1,3,3,5,5,3,4,13,13,12
91,GP,F,15,U,GT3,T,4,3,services,other,reputation,mother,1,1,0,no,no,yes,yes,yes,yes,yes,no,4,5,5,1,3,1,4,16,17,18
362,MS,F,18,U,GT3,T,3,3,services,services,other,mother,2,2,0,no,yes,no,no,yes,yes,yes,yes,4,3,2,1,3,3,0,11,11,10
104,GP,M,15,U,GT3,A,3,4,services,other,course,mother,1,2,0,no,yes,yes,yes,yes,yes,yes,no,5,4,4,1,1,1,0,16,18,18
18,GP,M,17,U,GT3,T,3,2,services,services,course,mother,1,1,3,no,yes,no,yes,yes,yes,yes,no,5,5,5,2,4,5,16,6,5,5
186,GP,M,16,U,GT3,T,1,2,services,services,other,mother,1,1,0,no,yes,yes,yes,yes,yes,yes,yes,3,3,3,1,2,3,2,11,12,11
374,MS,F,18,R,LE3,T,4,4,other,other,reputation,mother,2,3,0,no,no,no,no,yes,yes,yes,no,5,4,4,1,1,1,0,19,18,19
309,GP,F,19,U,LE3,T,1,1,at_home,other,reputation,other,1,2,1,yes,yes,no,yes,no,yes,yes,no,4,4,3,1,3,3,18,12,10,10


In [ ]:
features = data.columns[:-3]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoders = {}
x = []
for feature in features:
    if data[feature].dtype == "object":
        encoders[feature] = OneHotEncoder(sparse=False)
        encoded_feature = encoders[feature].fit_transform(data[feature].values.reshape(-1, 1))
    else:
        encoded_feature = data[feature].values.reshape(-1, 1)
    x.append(encoded_feature)

x = np.concatenate(x, axis=1)
y = data["G3"].values

### Splitting dataset

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=True)

### Logistic model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
Cs = [0.01, 0.1, 10, 100, 1000]

In [ ]:
params = {
    "C": Cs,
}

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  grid = GridSearchCV(LogisticRegression(), params, refit=True, verbose=3)

  grid.fit(x_train, y_train)
  y_pred = grid.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END ............................C=0.01;, score=0.169 total time=   0.1s
[CV 2/5] END ............................C=0.01;, score=0.225 total time=   0.1s
[CV 3/5] END ............................C=0.01;, score=0.239 total time=   0.1s
[CV 4/5] END ............................C=0.01;, score=0.225 total time=   0.1s
[CV 5/5] END ............................C=0.01;, score=0.197 total time=   0.1s
[CV 1/5] END .............................C=0.1;, score=0.127 total time=   0.1s
[CV 2/5] END .............................C=0.1;, score=0.155 total time=   0.1s
[CV 3/5] END .............................C=0.1;, score=0.211 total time=   0.1s
[CV 4/5] END .............................C=0.1;, score=0.183 total time=   0.1s
[CV 5/5] END .............................C=0.1;, score=0.183 total time=   0.1s
[CV 1/5] END ..............................C=10;, score=0.127 total time=   0.1s
[CV 2/5] END ..............................C=10;,

In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.449594,28.875000
1,0,-0.937249,43.375000
2,0,-1.138090,48.850000
3,0,-0.636898,37.975000
4,0,-1.007382,42.150000
5,0,-1.371279,43.153846
6,0,-0.736940,36.769231
7,0,-0.181586,23.743590
8,0,-0.798027,35.948718
9,0,-0.796744,35.923077


## Ridge Regression model for all attributes

### Load data and preprocess

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("student-mat.csv", sep=";")

In [ ]:
data.sample(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
21,GP,M,15,U,GT3,T,4,4,health,health,other,father,1,1,0,no,yes,yes,no,yes,yes,yes,no,5,4,2,1,1,5,0,12,15,15
329,GP,F,17,U,GT3,T,4,4,teacher,teacher,course,mother,2,3,0,no,yes,yes,no,no,yes,yes,yes,4,3,3,1,2,4,4,14,14,14
367,MS,F,17,R,GT3,T,1,1,other,services,reputation,mother,3,1,1,no,yes,yes,no,yes,yes,yes,yes,5,2,1,1,2,1,0,7,6,0
52,GP,M,15,U,LE3,A,4,2,health,health,other,father,2,1,1,no,no,no,no,yes,yes,no,no,5,5,5,3,4,5,6,11,11,10
249,GP,M,16,U,GT3,T,0,2,other,other,other,mother,1,1,0,no,no,yes,no,no,yes,yes,no,4,3,2,2,4,5,0,13,15,15
108,GP,M,15,R,GT3,T,4,4,other,other,home,father,4,4,0,no,yes,yes,yes,yes,yes,yes,yes,1,3,5,3,5,1,6,10,13,13
261,GP,M,18,U,GT3,T,4,3,teacher,other,course,mother,1,2,0,no,yes,yes,no,no,yes,yes,no,4,3,2,1,1,3,2,8,8,8
191,GP,F,17,U,GT3,T,1,1,at_home,services,course,mother,1,2,0,no,no,no,yes,yes,yes,yes,no,5,3,3,1,1,3,0,8,8,9
394,MS,M,19,U,LE3,T,1,1,other,at_home,course,father,1,1,0,no,no,no,no,yes,yes,yes,no,3,2,3,3,3,5,5,8,9,9
362,MS,F,18,U,GT3,T,3,3,services,services,other,mother,2,2,0,no,yes,no,no,yes,yes,yes,yes,4,3,2,1,3,3,0,11,11,10


In [ ]:
features = data.columns[:-3]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoders = {}
x = []
for feature in features:
    if data[feature].dtype == "object":
        encoders[feature] = OneHotEncoder(sparse=False)
        encoded_feature = encoders[feature].fit_transform(data[feature].values.reshape(-1, 1))
    else:
        encoded_feature = data[feature].values.reshape(-1, 1)
    x.append(encoded_feature)

x = np.concatenate(x, axis=1)
y = data["G3"].values

### Splitting dataset

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=True)

### Ridge Regression model

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
alphas = [0.01, 0.1, 10, 100, 1000]

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  ridge_model = make_pipeline(StandardScaler(), RidgeCV(alphas=alphas))

  ridge_model.fit(x_train, y_train)
  y_pred = ridge_model.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,0.065594,18.612782
1,0,0.154621,18.928045
2,0,0.282121,16.401745
3,0,0.247987,17.446242
4,0,0.207642,16.637544
5,0,0.009004,18.034692
6,0,0.224654,16.413266
7,0,-0.057752,21.255188
8,0,-0.079726,21.587428
9,0,0.238738,15.220242


## SGD Regression model for all attributes

### Load data and preprocess

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("student-mat.csv", sep=";")

In [ ]:
data.sample(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
45,GP,F,15,U,LE3,A,4,3,other,other,course,mother,1,2,0,yes,yes,yes,yes,yes,yes,yes,yes,5,2,2,1,1,5,8,8,8,6
369,MS,F,18,R,GT3,T,4,4,other,teacher,other,father,3,2,0,no,yes,yes,no,no,yes,yes,yes,3,2,2,4,2,5,10,14,12,11
66,GP,M,15,U,GT3,A,4,4,other,services,reputation,mother,1,4,0,no,yes,no,yes,no,yes,yes,yes,1,3,3,5,5,3,4,13,13,12
91,GP,F,15,U,GT3,T,4,3,services,other,reputation,mother,1,1,0,no,no,yes,yes,yes,yes,yes,no,4,5,5,1,3,1,4,16,17,18
362,MS,F,18,U,GT3,T,3,3,services,services,other,mother,2,2,0,no,yes,no,no,yes,yes,yes,yes,4,3,2,1,3,3,0,11,11,10
104,GP,M,15,U,GT3,A,3,4,services,other,course,mother,1,2,0,no,yes,yes,yes,yes,yes,yes,no,5,4,4,1,1,1,0,16,18,18
18,GP,M,17,U,GT3,T,3,2,services,services,course,mother,1,1,3,no,yes,no,yes,yes,yes,yes,no,5,5,5,2,4,5,16,6,5,5
186,GP,M,16,U,GT3,T,1,2,services,services,other,mother,1,1,0,no,yes,yes,yes,yes,yes,yes,yes,3,3,3,1,2,3,2,11,12,11
374,MS,F,18,R,LE3,T,4,4,other,other,reputation,mother,2,3,0,no,no,no,no,yes,yes,yes,no,5,4,4,1,1,1,0,19,18,19
309,GP,F,19,U,LE3,T,1,1,at_home,other,reputation,other,1,2,1,yes,yes,no,yes,no,yes,yes,no,4,4,3,1,3,3,18,12,10,10


In [ ]:
features = data.columns[:-3]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoders = {}
x = []
for feature in features:
    if data[feature].dtype == "object":
        encoders[feature] = OneHotEncoder(sparse=False)
        encoded_feature = encoders[feature].fit_transform(data[feature].values.reshape(-1, 1))
    else:
        encoded_feature = data[feature].values.reshape(-1, 1)
    x.append(encoded_feature)

x = np.concatenate(x, axis=1)
y = data["G3"].values

### Splitting dataset

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=True)

### SGD Regressor model

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
alphas = [0.01, 0.1, 10, 100, 1000]

In [ ]:
params = {
    "alpha": Cs,
}

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  grid = GridSearchCV(SGDRegressor(), params, refit=True, verbose=3)

  grid.fit(x_train, y_train)
  y_pred = grid.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END .......................alpha=0.01;, score=-0.119 total time=   0.0s
[CV 2/5] END .......................alpha=0.01;, score=-0.150 total time=   0.0s
[CV 3/5] END .......................alpha=0.01;, score=-0.888 total time=   0.0s
[CV 4/5] END .......................alpha=0.01;, score=-0.418 total time=   0.0s
[CV 5/5] END .......................alpha=0.01;, score=-0.401 total time=   0.0s
[CV 1/5] END ........................alpha=0.1;, score=-0.217 total time=   0.0s
[CV 2/5] END ........................alpha=0.1;, score=-0.188 total time=   0.0s
[CV 3/5] END .........................alpha=0.1;, score=0.132 total time=   0.0s
[CV 4/5] END .........................alpha=0.1;, score=0.017 total time=   0.0s
[CV 5/5] END ........................alpha=0.1;, score=-0.283 total time=   0.0s
[CV 1/5] END .........................alpha=10;, score=-0.203 total time=   0.0s
[CV 2/5] END .........................alpha=10;, 

In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.055230,21.019520
1,0,-0.072279,24.008317
2,0,-0.003532,22.928199
3,0,0.002187,23.148646
4,0,-0.003122,21.063047
5,0,-0.022559,18.609097
6,0,-0.039369,22.002360
7,0,0.000138,20.091898
8,0,-0.052768,21.048433
9,0,0.000147,19.990482


##Features selection for training model

In [20]:
selected_featues = []
import json
with open('/content/StudentPerformance/effect_single_attr_mat.json') as f:
  json_file = json.load(f)
for i in range(len(json_file)):
  feature = json_file[i]['attribute']
  selected_featues.append(feature)

In [21]:
selected_featues

['sex',
 'age',
 'address',
 'Medu',
 'Fedu',
 'Mjob',
 'failures',
 'paid',
 'higher',
 'romantic',
 'goout',
 'absences']

In [22]:
processed_data = data[selected_featues]

In [23]:
#process data by one hot
encoders = {}
x = []
for feature in selected_featues:
    if processed_data[feature].dtype == "object":
        encoders[feature] = OneHotEncoder(sparse=False)
        encoded_feature = encoders[feature].fit_transform(processed_data[feature].values.reshape(-1, 1))
    else:
        encoded_feature = processed_data[feature].values.reshape(-1, 1)
    x.append(encoded_feature)

x = np.concatenate(x, axis=1)
y = data["G3"].values

###Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
linear_models = {}
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  ln_model = Pipeline([
                     ("scaler", StandardScaler()),
                     ("linear_model", LinearRegression())
  ])

  ln_model.fit(x_train, y_train)
  y_pred = ln_model.predict(x_test)
  
  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

  linear_models[f"Stage_{ith}"] = ln_model

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,0.233728,15.263659
1,0,0.128362,19.515971
2,0,0.103307,20.487190
3,0,0.182731,18.960127
4,0,0.164822,17.536656
5,0,0.113114,16.140038
6,0,0.071985,19.645127
7,0,0.238990,15.292242
8,0,0.218087,15.633126
9,0,0.008528,19.822920


###Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
n_estimators = [100, 1000]

In [ ]:
params = {
    "n_estimators": n_estimators,
}

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  grid = GridSearchCV(RandomForestRegressor(), params, refit=True, verbose=3)

  grid.fit(x_train, y_train)
  y_pred = grid.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .................n_estimators=100;, score=-0.251 total time=   0.3s
[CV 2/5] END ..................n_estimators=100;, score=0.248 total time=   0.3s
[CV 3/5] END ..................n_estimators=100;, score=0.191 total time=   0.3s
[CV 4/5] END ..................n_estimators=100;, score=0.180 total time=   0.3s
[CV 5/5] END .................n_estimators=100;, score=-0.015 total time=   0.3s
[CV 1/5] END ................n_estimators=1000;, score=-0.234 total time=   2.9s
[CV 2/5] END .................n_estimators=1000;, score=0.263 total time=   2.3s
[CV 3/5] END .................n_estimators=1000;, score=0.230 total time=   2.0s
[CV 4/5] END .................n_estimators=1000;, score=0.143 total time=   1.9s
[CV 5/5] END .................n_estimators=1000;, score=0.017 total time=   1.9s
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .................n_estimators=100;, score=-0.430 total ti

In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,0.150803,16.915471
1,0,0.284809,16.013123
2,0,0.269804,16.683146
3,0,0.102392,20.823944
4,0,0.135739,18.147317
5,0,0.170032,15.104209
6,0,0.360819,13.530801
7,0,-0.175421,23.619695
8,0,-0.018784,20.368988
9,0,0.498354,10.029627


###Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  dc_model = DecisionTreeRegressor()

  dc_model.fit(x_train, y_train)
  y_pred = dc_model.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.190110,23.706250
1,0,-0.550916,34.725000
2,0,-0.023088,23.375000
3,0,-0.094857,25.400000
4,0,-0.001310,21.025000
5,0,-0.994030,36.288462
6,0,-0.466830,31.051282
7,0,-0.510797,30.358974
8,0,-0.837784,36.743590
9,0,-0.549227,30.974359


###Extra Tree model

In [ ]:
from sklearn.tree import ExtraTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  dc_model = ExtraTreeRegressor()

  dc_model.fit(x_train, y_train)
  y_pred = dc_model.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.922751,38.300000
1,0,-0.997544,44.725000
2,0,-0.905022,43.525000
3,0,-0.222016,28.350000
4,0,-0.585903,33.300000
5,0,-0.459682,26.564103
6,0,-0.415958,29.974359
7,0,-0.132141,22.750000
8,0,-0.294015,25.871795
9,0,-0.606939,32.128205


###Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
Cs = [0.01, 0.1, 10, 100, 1000]
params = {
    "C": Cs,
}

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  grid = GridSearchCV(LogisticRegression(), params, refit=True, verbose=3)

  grid.fit(x_train, y_train)
  y_pred = grid.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END ............................C=0.01;, score=0.282 total time=   0.1s
[CV 2/5] END ............................C=0.01;, score=0.211 total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[CV 3/5] END ............................C=0.01;, score=0.225 total time=   0.1s
[CV 4/5] END ............................C=0.01;, score=0.254 total time=   0.1s
[CV 5/5] END ............................C=0.01;, score=0.197 total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[CV 2/5] END ............................C=1000;, score=0.113 total time=   0.1s
[CV 3/5] END ............................C=1000;, score=0.211 total time=   0.1s
[CV 4/5] END ............................C=1000;, score=0.127 total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[CV 5/5] END ............................C=1000;, score=0.239 total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.907690,38.000000
1,0,-0.836757,41.125000
2,0,-1.191706,50.075000
3,0,-0.869662,43.375000
4,0,-1.262174,47.500000
5,0,-0.911958,34.794872
6,0,-0.579477,33.435897
7,0,-0.120338,22.512821
8,0,-0.682604,33.641026
9,0,-0.513318,30.256410


###Ridge Regression

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
alphas = [0.01, 0.1, 10, 100, 1000]

In [ ]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  ridge_model = make_pipeline(StandardScaler(), RidgeCV(alphas=alphas))

  ridge_model.fit(x_train, y_train)
  y_pred = ridge_model.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,0.135331,17.223663
1,0,0.027834,21.766792
2,0,0.185608,18.606823
3,0,0.187901,18.840195
4,0,0.158159,17.676562
5,0,0.093263,16.501297
6,0,0.122967,18.565889
7,0,0.284511,14.377520
8,0,0.145825,17.077888
9,0,0.151494,16.964550


###SGD Regressor model

In [16]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [17]:
alphas = [0.01, 0.1, 10, 100, 1000]

In [18]:
params = {
    "alpha": Cs,
}

NameError: ignored

In [19]:
results = {
    "Stage": [],
    "R2": [],
    "MSE": []
}

In [ ]:
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  grid = GridSearchCV(SGDRegressor(), params, refit=True, verbose=3)

  grid.fit(x_train, y_train)
  y_pred = grid.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END .......................alpha=0.01;, score=-0.256 total time=   0.0s
[CV 2/5] END .......................alpha=0.01;, score=-0.125 total time=   0.0s
[CV 3/5] END ........................alpha=0.01;, score=0.090 total time=   0.0s
[CV 4/5] END .......................alpha=0.01;, score=-0.530 total time=   0.0s
[CV 5/5] END .......................alpha=0.01;, score=-0.704 total time=   0.0s
[CV 1/5] END .........................alpha=0.1;, score=0.080 total time=   0.0s
[CV 2/5] END .........................alpha=0.1;, score=0.160 total time=   0.0s
[CV 3/5] END ........................alpha=0.1;, score=-0.278 total time=   0.0s
[CV 4/5] END ........................alpha=0.1;, score=-0.386 total time=   0.0s
[CV 5/5] END ........................alpha=0.1;, score=-0.329 total time=   0.0s
[CV 1/5] END .........................alpha=10;, score=-1.055 total time=   0.0s
[CV 2/5] END .........................alpha=10;, 

In [ ]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,-0.006017,20.039235
1,0,0.075792,20.693013
2,0,-0.035081,23.649019
3,0,-0.095743,25.420561
4,0,-0.041814,21.875500
5,0,-0.006755,18.321479
6,0,-0.087626,23.023910
7,0,-0.112335,22.352005
8,0,-0.001424,20.021891
9,0,-0.277666,25.544921


###SVM

In [25]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.01,0.1, 1],
              'gamma': [ 0.1, 0.01,1],
              'kernel': [ 'rbf']}
ith = 0
for train_index, test_index in folds.split(x, y):
  x_train = x[train_index]
  y_train = y[train_index]
  
  x_test = x[test_index]
  y_test = y[test_index]

  grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=3)

  grid.fit(x_train, y_train)
  y_pred = grid.predict(x_test)

  results["Stage"].append(ith)
  results["R2"].append(r2_score(y_test, y_pred))
  results["MSE"].append(mean_squared_error(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ....C=0.01, gamma=0.1, kernel=rbf;, score=-0.282 total time=   0.0s
[CV 2/5] END ....C=0.01, gamma=0.1, kernel=rbf;, score=-0.012 total time=   0.0s
[CV 3/5] END ....C=0.01, gamma=0.1, kernel=rbf;, score=-0.037 total time=   0.0s
[CV 4/5] END ....C=0.01, gamma=0.1, kernel=rbf;, score=-0.017 total time=   0.0s
[CV 5/5] END ....C=0.01, gamma=0.1, kernel=rbf;, score=-0.056 total time=   0.0s
[CV 1/5] END ...C=0.01, gamma=0.01, kernel=rbf;, score=-0.276 total time=   0.0s
[CV 2/5] END ...C=0.01, gamma=0.01, kernel=rbf;, score=-0.011 total time=   0.0s
[CV 3/5] END ...C=0.01, gamma=0.01, kernel=rbf;, score=-0.038 total time=   0.0s
[CV 4/5] END ...C=0.01, gamma=0.01, kernel=rbf;, score=-0.019 total time=   0.0s
[CV 5/5] END ...C=0.01, gamma=0.01, kernel=rbf;, score=-0.060 total time=   0.0s
[CV 1/5] END ......C=0.01, gamma=1, kernel=rbf;, score=-0.303 total time=   0.0s
[CV 2/5] END ......C=0.01, gamma=1, kernel=rbf;, 

In [26]:
results = pd.DataFrame(results)
results

,Stage,R2,MSE
0,0,0.098901,17.949319
1,0,0.012415,22.112025
2,0,0.153649,19.337001
3,0,0.128903,20.208908
4,0,0.044155,20.070351
5,0,0.121206,15.992783
6,0,0.127508,18.469746
7,0,0.291512,14.236843
8,0,0.194364,16.107430
9,0,0.169695,16.600646
